In [1]:
import torch
print(torch.__version__)

1.8.0+cpu


/home/mahshid/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os 
import pandas as pd 
import numpy as np
import time
import torch 
import torchcsprng as prng

import torch
import numpy as np
from opacus import PrivacyEngine
from tqdm import tqdm
from model           import MLP
from sklearn.metrics import roc_curve, confusion_matrix
from typing          import Tuple, List
from dataloader      import DatasetLoader
from tqdm            import tqdm
from opacus          import PrivacyEngine
from inputs          import args
from sklearn.metrics import classification_report

In [4]:
import pandas as pd 
import numpy as np
import torchcsprng as prng


from typing                       import Tuple, List
from sklearn.preprocessing import StandardScaler
#from sklearn.preprocessing        import RobustScaler
from sklearn.utils import resample
from torch.utils.data             import DataLoader
from opacus.utils.uniform_sampler import UniformWithReplacementSampler


class DatasetLoader:

    def __init__(self, df: pd.DataFrame, num_clients:int, sample_rate:float, weights:List[float]=None) -> None:
        
        self._num_clients  = num_clients
        (self._X_train, self._y_train, self._X_test, self._y_test) = self._load_train_test_data(df)
        #self._generator    = prng.create_random_device_generator("/dev/urandom")
        self._sample_rate  = sample_rate
        self._clients_data = self._assign_data_to_clients(self._X_train, self._y_train, weights)


    @property
    def clients_data(self) -> dict:
        return self._clients_data

    def get_train_data(self) -> DataLoader:
        train_dataset = [(torch.tensor(self._X_train[i], dtype=torch.float32), torch.tensor(self._y_train[i], dtype=torch.float32)) for i in range(len(self._X_train))]
        return DataLoader(train_dataset, batch_size=64, num_workers=1, pin_memory=False)
     
        
    def get_test_data(self) -> DataLoader:
        test_dataset = [(torch.tensor(self._X_test[i], dtype=torch.float32), torch.tensor(self._y_test[i], dtype=torch.float32)) for i in range(len(self._X_test))]
        return DataLoader(test_dataset, batch_size=64, num_workers=1, pin_memory=False)

    def _load_train_test_data(self, df:pd.DataFrame) -> Tuple[np.ndarray, np.array, np.ndarray, np.array]:
      
        '''
        count_class_0, count_class_1 = df.Label.value_counts()

        df_class_0 = df[df['Label'] == 0]
        df_class_1 = df[df['Label'] == 1]

        df_class_1_over = df_class_1.sample(count_class_0, replace=True)
        df = pd.concat([df_class_0, df_class_1_over], axis=0)
        '''
     
        attack_indices     = df.loc[df.Label == 1].index
        normal_indices = df.loc[df.Label == 0].index
        
       
        attack_instances=np.random.random(len(attack_indices))<0.80
        normal_instances=np.random.random(len(normal_indices))<0.80


       
        test_attack = df.iloc[attack_indices[~attack_instances]]
        test_normal = df.iloc[normal_indices[~normal_instances]]
        test_df = pd.concat([test_attack, test_normal])

        
        train_attack = df.iloc[attack_indices[attack_instances]]
        train_normal = df.iloc[normal_indices[normal_instances]]
        train_df = pd.concat([train_attack, train_normal])

        train_df = train_df.sample(frac=1).reset_index(drop=True)
        test_df  = test_df.sample(frac=1).reset_index(drop=True)


        attack = train_df.loc[train_df.Label == 1]
        normal = train_df.loc[train_df.Label == 0][:len(attack)]
        train_df_even = pd.concat([attack, normal])
        train_df_even = train_df_even.sample(frac=1).reset_index(drop=True)


        X_train = train_df_even.drop('Label', axis=1).values
        y_train = train_df_even.Label.values

        X_test = test_df.drop('Label', axis=1).values
        y_test = test_df.Label.values

        scaling = StandardScaler()
        X_train = scaling.fit_transform(X_train)
        X_test = scaling.transform(X_test)
        
        return X_train, y_train, X_test, y_test

    def _assign_data_to_clients(self, X_train:np.ndarray, y_train:np.ndarray, weights:List[float]) -> dict:
        
            num_samples_per_client = X_train.shape[0] // self._num_clients
            clients_all_data = {}
            
            for i in range(self._num_clients):
                batched_x = self._X_train[i*num_samples_per_client:(i+1)*num_samples_per_client]
                batched_y = self._y_train[i*num_samples_per_client:(i+1)*num_samples_per_client]

                client_dataset = [(batched_x[i], batched_y[i]) for i in range(len(batched_x))]
                data = DataLoader(client_dataset, batch_size=64, 
                                  num_workers=1, pin_memory=False)
           
                clients_all_data[f'client_{i}'] = data 
        #else:
            # TODO
            #clients_all_data = {}
            return clients_all_data

In [5]:
MAX_GRAD_NORM = 1.
EPSILON = 20.0
#DELTA = 1e-7
DELTA = 1e-4
EPOCHS = 6
#NOISE= 1.3
NOISE= 0.5

In [6]:
class Server:
 
    def __init__(self):
        self._model = MLP(26,2)
        #self._model = MLP(6,2)

        # client_parameters[i] is the parameters of model trained by client i
        self._client_parameters        = []
        # client_samples_processed[i] is the number of samples client i used for training
        self._client_samples_processed = []
        
    @property
    def parameters(self) -> torch.nn.Parameter:
        return self._model.parameters()

    @property 
    def model(self):
        return self._model

    def add_client_parameters(self, parameters: torch.nn.Parameter, num_samples: int):
        self._client_parameters.append(parameters)
        self._client_samples_processed.append(num_samples)
    
    def _weight_scaling_factor(self, i:int) -> float:
     
        global_count = sum(self._client_samples_processed)
        local_count  = self._client_samples_processed[i]
        return local_count / global_count

    def _scale_model_parameters(self, i:int) -> None:
        
        scalar     = self._weight_scaling_factor(i)
        parameters = list(self._client_parameters[i])
        new_parameters = [parameters[i]*scalar for i in range(len(parameters))]
        with torch.no_grad():
            for idx, (param, new_param) in enumerate(zip(self._client_parameters[i], new_parameters)):
                self._client_parameters[i][idx] = new_param

    def aggregate_parameters(self) -> None:
    
        for i in range(len(self._client_parameters)):
            self._scale_model_parameters(i)


        aggregated_parameters = [torch.zeros(list(self._client_parameters[0])[j].shape) 
                                for j in range(len(list(self._client_parameters[0])))]

        for j in range(len(list(self._client_parameters[0]))):
            for i in range(len(self._client_parameters)):
                aggregated_parameters[j] += self._client_parameters[i][j]
        
        with torch.no_grad():
            for model_param, new_param in zip(self._model.parameters(), aggregated_parameters):
                model_param.data = new_param.view_as(model_param.data)
        
        self._client_parameters        = []
        self._client_samples_processed = []

class Client:
    
    def __init__(self, train_loader, noise_multiplier, max_grad_norm, DELTA):
        self._model      = MLP(26,2)
        #self._model = MLP(6,2)
        self._lr         = args.lr
        self._loss       = torch.nn.CrossEntropyLoss()
        #self._optimizer  = torch.optim.SGD(self._model.parameters(), self._lr)
     
        self.train_loader= train_loader
        self.privacy_engine = PrivacyEngine(secure_mode=False)
        self._optimizer  = torch.optim.SGD(self._model.parameters(), self._lr)
        self._model, self._optimizer, self.train_loader = self.privacy_engine.make_private(
            module=self._model,
            optimizer= self._optimizer,
            data_loader=self.train_loader,
            noise_multiplier= NOISE,
            max_grad_norm=MAX_GRAD_NORM,
        )
        
    @property
    def model(self):
        return self._model
    
    @property
    def parameters(self):
        return self._model.parameters()
    
    def train_one_epoch(self, train_loader: DataLoader, device:torch.device, epoch:int, client:str) -> None:
        losses = []
        self._model.train()
        for x, y in tqdm(train_loader):
            x, y = x.to(device).to(torch.float32), y.to(device)
            #x, y = x.to(device).to(torch.long), y.to(device).to(torch.long)
            yhat = self._model(x)
            self._optimizer.zero_grad()
            loss = self._loss(yhat, y.to(torch.long))
            loss.backward()
            losses.append(loss.item())
            self._optimizer.step()
        epsilon = self.privacy_engine.get_epsilon(DELTA)
        print(
            f"Client: {client} "
            f"Train Epoch: {epoch} \t"
            f"Loss: {np.mean(losses):.6f} "
            f" | (ε = {epsilon:.2f})"
        )
        
    def update_parameters(self, new_parameters: List[torch.Tensor]):
        with torch.no_grad():
            for model_param, new_param in zip(self._model.parameters(), new_parameters):
                model_param.data = new_param.view_as(model_param.data)

In [8]:
def evaluate(model, device, test_loader):
    model.eval()
    loss = torch.nn.CrossEntropyLoss()
    test_loss = 0
    correct   = 0

    predarray  = torch.zeros(len(test_loader.dataset))
    probsarray = torch.zeros((len(test_loader.dataset),2))
    actual     = torch.zeros(len(test_loader.dataset))
    with torch.no_grad():
        for batch_idx, (x, y) in enumerate(tqdm(test_loader)):
            #x, y = x.to(device), y.to(device)
            x, y = x.to(device).to(torch.float32), y.to(device)
            #x, y = x.to(device).to(torch.long), y.to(device).to(torch.long)
            yhat = model(x)
            
            test_loss    += loss(yhat, y.to(torch.long)).item()
            probabilities = torch.nn.functional.softmax(yhat,1)
            predictions   = torch.argmax(probabilities, 1)
            correct      += predictions.eq(y.view_as(predictions)).sum().item()

            B = 64
            predarray[batch_idx*B:min(len(test_loader.dataset), (batch_idx+1)*B)]  = predictions
            probsarray[batch_idx*B:min(len(test_loader.dataset), (batch_idx+1)*B)] = probabilities
            actual[batch_idx*B:min(len(test_loader.dataset), (batch_idx+1)*B)]     = y

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return correct / len(test_loader.dataset), predarray, probsarray, actual

def plot_confusion_matrix(labels:torch.tensor, predictions:torch.tensor, savefile:str) -> None:
    print("Confusion matrix on test data:")
    print(confusion_matrix(labels, predictions))
    print('Classification Report:')
    print(classification_report(labels, predictions))

In [9]:
import torch
from model import MLP
from dataclasses import dataclass

@dataclass
class parameters:
    lr: float              = 0.01
    device: torch.device   = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    sample_rate: float     = 0.2
    #noise_multiplier:float = 1.3
    noise_multiplier:float = 0.5
    max_grad_norm:float    = 1.1
    #DELTA:float     = 1e-7
    DELTA:float     = 1e-4
args = parameters()

In [10]:
csv_file         = os.path.join("/home/mahshid/python-workspace/clean_wustl1111.csv")
device           = args.device
sample_rate      = args.sample_rate
df           = pd.read_csv(csv_file)
dataloader   = DatasetLoader(df=df, num_clients=2, sample_rate=sample_rate)
train_loader = dataloader.get_train_data()
test_loader  = dataloader.get_test_data()

In [12]:
#DNN Model
class MLP(torch.nn.Module):
    def __init__(self, input_shape, num_classes):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_shape, 64)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(64,32)
        self.relu2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(32, 2)
        self.relu3 = torch.nn.ReLU()
        #self.dropout = torch.nn.Dropout(0.5)
        #self.fc4 = torch.nn.Linear(32, 16)
        #self.relu4 = torch.nn.ReLU()
        
        #self.fc5 = torch.nn.Linear(16, 2)
        self.Softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
#         x = self.dropout(x)
        #x = self.fc4(x)
        #x = self.relu4(x)
        #x = self.fc5(x)
        x = self.Softmax(x)
        return x

input_shape = 27  # number of features

In [11]:
#CNN Model
class MLP(torch.nn.Module):
    def __init__(self, input_shape, num_classes):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_shape, 128)
        self.relu1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(128,64)
        self.relu2 = torch.nn.ReLU()
        self.fc3 = torch.nn.Linear(64, 32)
        self.relu3 = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(0.5)
        self.fc4 = torch.nn.Linear(32, 16)
        self.relu4 = torch.nn.ReLU()
        
        self.fc5 = torch.nn.Linear(16, 2)
        self.Softmax = torch.nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
#         x = self.dropout(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.fc5(x)
        x = self.Softmax(x)
        return x

input_shape = 27  # number of features

In [12]:
# FL with differential privacy
clients_data = dataloader.clients_data
clients = {
            client: Client(train_loader,
                           noise_multiplier=1.5, 
                           max_grad_norm=1, DELTA=1e-4) 
            for client in clients_data
        }
server  = Server()
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()

accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
roc_nopriv = roc_curve(labels, probabilities[:,1])
plot_confusion_matrix(labels, predictions, 'federated_privacy.png')
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

/home/mahshid/.local/lib/python3.8/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
  0%|          | 0/640 [00:00<?, ?it/s]

/home/mahshid/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 640/640 [00:05<00:00, 116.35it/s]
/home/mahshid/.local/lib/python3.8/site-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)


Client: client_0 Train Epoch: 0 	Loss: 0.554594  | (ε = 0.04)


100%|██████████| 640/640 [00:05<00:00, 120.22it/s]


Client: client_1 Train Epoch: 0 	Loss: 0.442507  | (ε = 0.04)


...

Client: client_0 Train Epoch: 99 	Loss: 0.380661  | (ε = 0.44)


100%|██████████| 640/640 [00:05<00:00, 112.64it/s]


Client: client_1 Train Epoch: 99 	Loss: 0.380100  | (ε = 0.44)


100%|██████████| 427/427 [00:01<00:00, 299.41it/s]



Test set: Average loss: 0.0059, Accuracy: 25441/27297 (93.20%)

Confusion matrix on test data:
[[ 1394  1510]
 [  346 24047]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.48      0.60      2904
         1.0       0.94      0.99      0.96     24393

    accuracy                           0.93     27297
   macro avg       0.87      0.73      0.78     27297
weighted avg       0.93      0.93      0.92     27297


Training time: 6.723403930664062e-05

Test time: 1.9073486328125e-05

TIME: 0.0002434253692626953 seconds


In [13]:
# FL with differential privacy,  noise_multiplier=0.5
clients_data = dataloader.clients_data
clients = {
            client: Client(train_loader,
                           noise_multiplier=0.5, 
                           max_grad_norm=1, DELTA=1e-4) 
            for client in clients_data
        }
server  = Server()
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()

accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
roc_nopriv = roc_curve(labels, probabilities[:,1])
plot_confusion_matrix(labels, predictions, 'federated_privacy.png')
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

/home/mahshid/.local/lib/python3.8/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
  0%|          | 0/681 [00:00<?, ?it/s]/home/mahshid/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 681/681 [00:06<00:00, 108.86it/s]
/home/mahshid/.local/lib/python3.8/site-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in 

Client: client_0 Train Epoch: 0 	Loss: 0.530005  | (ε = 1.58)


100%|██████████| 681/681 [00:05<00:00, 121.00it/s]


Client: client_1 Train Epoch: 0 	Loss: 0.439987  | (ε = 1.58)


100%|██████████| 681/681 [00:06<00:00, 112.99it/s]


...

Client: client_0 Train Epoch: 99 	Loss: 0.381529  | (ε = 6.69)


100%|██████████| 681/681 [00:05<00:00, 117.38it/s]


Client: client_1 Train Epoch: 99 	Loss: 0.382990  | (ε = 6.69)


100%|██████████| 344/344 [00:01<00:00, 277.04it/s]



Test set: Average loss: 0.0060, Accuracy: 20386/21983 (92.74%)

Confusion matrix on test data:
[[ 1093  1294]
 [  303 19293]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.78      0.46      0.58      2387
         1.0       0.94      0.98      0.96     19596

    accuracy                           0.93     21983
   macro avg       0.86      0.72      0.77     21983
weighted avg       0.92      0.93      0.92     21983


Training time: 5.6743621826171875e-05

Test time: 2.3126602172851562e-05

TIME: 0.00022125244140625 seconds


In [16]:
# differential privacy + CNN
clients_data = dataloader.clients_data
clients = {
            client: Client(train_loader,
                           noise_multiplier=1.5, 
                           max_grad_norm=1, DELTA=1e-4) 
            for client in clients_data
        }
server  = Server()
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()

accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
roc_nopriv = roc_curve(labels, probabilities[:,1])
plot_confusion_matrix(labels, predictions, 'federated_privacy.png')
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

/home/mahshid/.local/lib/python3.8/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
  0%|          | 0/682 [00:00<?, ?it/s]

/home/mahshid/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 682/682 [00:09<00:00, 69.98it/s]
/home/mahshid/.local/lib/python3.8/site-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)


Client: client_0 Train Epoch: 0 	Loss: 0.544264  | (ε = 0.04)


100%|██████████| 682/682 [00:08<00:00, 76.69it/s]


Client: client_1 Train Epoch: 0 	Loss: 0.445654  | (ε = 0.04)


...

Client: client_0 Train Epoch: 99 	Loss: 0.382139  | (ε = 0.43)


100%|██████████| 682/682 [00:09<00:00, 73.83it/s]


Client: client_1 Train Epoch: 99 	Loss: 0.382224  | (ε = 0.43)


100%|██████████| 341/341 [00:01<00:00, 314.20it/s]



Test set: Average loss: 0.0060, Accuracy: 20308/21814 (93.10%)

Confusion matrix on test data:
[[ 1026  1285]
 [  221 19282]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.82      0.44      0.58      2311
         1.0       0.94      0.99      0.96     19503

    accuracy                           0.93     21814
   macro avg       0.88      0.72      0.77     21814
weighted avg       0.93      0.93      0.92     21814


Training time: 5.817413330078125e-05

Test time: 1.9311904907226562e-05

TIME: 0.0002384185791015625 seconds


In [12]:
# differential privacy + CNN, noise_multiplier=0.5
clients_data = dataloader.clients_data
clients = {
            client: Client(train_loader,
                           noise_multiplier=0.5, 
                           max_grad_norm=1, DELTA=1e-4) 
            for client in clients_data
        }
server  = Server()
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()

accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
roc_nopriv = roc_curve(labels, probabilities[:,1])
plot_confusion_matrix(labels, predictions, 'federated_privacy.png')
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

/home/mahshid/.local/lib/python3.8/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
  0%|          | 0/681 [00:00<?, ?it/s]

/home/mahshid/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 681/681 [00:09<00:00, 71.26it/s]
/home/mahshid/.local/lib/python3.8/site-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)


Client: client_0 Train Epoch: 0 	Loss: 0.557040  | (ε = 1.58)


100%|██████████| 681/681 [00:09<00:00, 72.40it/s]


Client: client_1 Train Epoch: 0 	Loss: 0.460150  | (ε = 1.58)


...

100%|██████████| 681/681 [00:09<00:00, 75.03it/s]


Client: client_1 Train Epoch: 99 	Loss: 0.420595  | (ε = 6.69)


100%|██████████| 344/344 [00:01<00:00, 269.20it/s]



Test set: Average loss: 0.0066, Accuracy: 19636/21955 (89.44%)

Confusion matrix on test data:
[[    0  2319]
 [    0 19636]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      2319
         1.0       0.89      1.00      0.94     19636

    accuracy                           0.89     21955
   macro avg       0.45      0.50      0.47     21955
weighted avg       0.80      0.89      0.84     21955


Training time: 5.459785461425781e-05

Test time: 1.6689300537109375e-05

TIME: 0.0002338886260986328 seconds


/home/mahshid/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahshid/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mahshid/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# centralized learning

In [55]:
class Server:
    def __init__(self):
        self._model = MLP(26,2)

        # client_parameters[i] is the parameters of model trained by client i
        self._client_parameters        = []
        # client_samples_processed[i] is the number of samples client i used for training
        self._client_samples_processed = []
        
    @property
    def parameters(self) -> torch.nn.Parameter:
        return self._model.parameters()

    @property 
    def model(self):
        return self._model

    def add_client_parameters(self, parameters: torch.nn.Parameter, num_samples: int):
        self._client_parameters.append(parameters)
        self._client_samples_processed.append(num_samples)
    
    def _weight_scaling_factor(self, i:int) -> float:

        global_count = sum(self._client_samples_processed)
        local_count  = self._client_samples_processed[i]
        return local_count / global_count

    def _scale_model_parameters(self, i:int) -> None:
 
        scalar     = self._weight_scaling_factor(i)
        parameters = list(self._client_parameters[i])
        new_parameters = [parameters[i]*scalar for i in range(len(parameters))]
        with torch.no_grad():
            for idx, (param, new_param) in enumerate(zip(self._client_parameters[i], new_parameters)):
                self._client_parameters[i][idx] = new_param

    def aggregate_parameters(self) -> None:
 
        # scale weights of all clients
        for i in range(len(self._client_parameters)):
            self._scale_model_parameters(i)


        aggregated_parameters = [torch.zeros(list(self._client_parameters[0])[j].shape) 
                                for j in range(len(list(self._client_parameters[0])))]

        for j in range(len(list(self._client_parameters[0]))):
            for i in range(len(self._client_parameters)):
                aggregated_parameters[j] += self._client_parameters[i][j]
        
        with torch.no_grad():
            for model_param, new_param in zip(self._model.parameters(), aggregated_parameters):
                model_param.data = new_param.view_as(model_param.data)
        
        # after an update, we requery every client
        self._client_parameters        = []
        self._client_samples_processed = []

class Client:

    def __init__(self):
        self._model      = MLP(26,2)
        self._lr         = args.lr
        self._loss       = torch.nn.CrossEntropyLoss()
        self._optimizer  = torch.optim.SGD(self._model.parameters(), self._lr)
        
    @property
    def model(self):
        return self._model
    
    @property
    def parameters(self):
        return self._model.parameters()
    
    def train_one_epoch(self, train_loader: DataLoader, device:torch.device, epoch:int, client:str) -> None:

        losses = []
        self._model.train()
        for x, y in tqdm(train_loader):
            x, y = x.to(device), y.to(device)
            yhat = self._model(x.float())
            self._optimizer.zero_grad()
            loss = self._loss(yhat, y)
            loss.backward()
            losses.append(loss.item())
            self._optimizer.step()

        print(
                f"Client: {client} " 
                f"Train Epoch: {epoch} \t"
                f"Loss: {np.mean(losses):.6f} "
            )
        
    def update_parameters(self, new_parameters: List[torch.Tensor]):
   
        with torch.no_grad():
            for model_param, new_param in zip(self._model.parameters(), new_parameters):
                model_param.data = new_param.view_as(model_param.data)

In [ ]:
# no differential privacy + DNN
clients_data = dataloader.clients_data
clients = {
            client: Client() 
            for client in clients_data
        }
server  = Server()
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()
accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
roc_nopriv = roc_curve(labels, probabilities[:,1])
plot_confusion_matrix(labels, predictions, 'federated_no_privacy.png')
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

In [61]:
# no differential privacy + CNN
clients_data = dataloader.clients_data
clients = {
            client: Client() 
            for client in clients_data
        }
server  = Server()
for epoch in range(100):
    for i, (client, client_dataloader) in enumerate(clients_data.items()):
        clients[client].update_parameters(server.parameters)
        clients[client].train_one_epoch(client_dataloader, device, epoch, client)
        server.add_client_parameters(clients[client].parameters, len(client_dataloader.dataset))
    server.aggregate_parameters()
accuracy, predictions, probabilities, labels = evaluate(server.model, device, test_loader)
roc_nopriv = roc_curve(labels, probabilities[:,1])
plot_confusion_matrix(labels, predictions, 'federated_no_privacy.png')
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

  0%|          | 0/643 [00:00<?, ?it/s]

100%|██████████| 643/643 [00:03<00:00, 200.46it/s]


Client: client_0 Train Epoch: 0 	Loss: 0.521048 


100%|██████████| 643/643 [00:03<00:00, 207.12it/s]


Client: client_1 Train Epoch: 0 	Loss: 0.444010 


...

Client: client_0 Train Epoch: 99 	Loss: 0.370587 


100%|██████████| 643/643 [00:02<00:00, 215.16it/s]


Client: client_1 Train Epoch: 99 	Loss: 0.371192 


100%|██████████| 421/421 [00:01<00:00, 243.07it/s]



Test set: Average loss: 0.0058, Accuracy: 25386/26900 (94.37%)

Confusion matrix on test data:
[[ 1495  1296]
 [  218 23891]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.87      0.54      0.66      2791
         1.0       0.95      0.99      0.97     24109

    accuracy                           0.94     26900
   macro avg       0.91      0.76      0.82     26900
weighted avg       0.94      0.94      0.94     26900


Training time: 0.00010657310485839844

Test time: 2.9087066650390625e-05

TIME: 0.00035452842712402344 seconds
